# -----------------------------------------__E__xtract-----------------------------------------

In [92]:
# Dependencies
import requests
import pandas as pd 
from datetime import datetime
from sqlalchemy import create_engine
from config import EIA_API_Key
import json
import numpy as np
from functools import reduce

In [93]:
datetime(2015, 1, 1)

datetime.datetime(2015, 1, 1, 0, 0)

## __Source Scraping: Following the API calling we then created a dataframe in pandas__

In [94]:
# Source 1: Gasoline supplied per week.
# Source 1: Calling the API
url = "http://api.eia.gov/series/?api_key=d13b79ac20ecbc443a35c43f026bbd48&series_id=PET.WGFUPUS2.W"
gasoline = requests.get(url).json()

gas_df = pd.json_normalize(gasoline, ['series',['data']],  errors='ignore', max_level=1)
gas_df.columns = ['Date','gasVolume']
gas_df

,Date,gasVolume
0,20201009,8576
1,20201002,8896
2,20200925,8529
3,20200918,8515
4,20200911,8478
...,...,...
1544,19910308,6875
1545,19910301,7224
1546,19910222,6582
1547,19910215,6433


In [95]:
gas_df['Date'] =  pd.to_datetime(gas_df['Date'])
gas_df['Year'] = gas_df.Date.dt.year
gas_df['Month'] = gas_df.Date.dt.month
gas_df.head()

,Date,gasVolume,Year,Month
0,2020-10-09,8576,2020,10
1,2020-10-02,8896,2020,10
2,2020-09-25,8529,2020,9
3,2020-09-18,8515,2020,9
4,2020-09-11,8478,2020,9


In [96]:
gas_df = gas_df.groupby(['Year','Month']).min()
gas_df = gas_df.reset_index()
gas_df.head()

,Year,Month,Date,gasVolume
0,1991,2,1991-02-08,6433
1,1991,3,1991-03-01,6777
2,1991,4,1991-04-05,6916
3,1991,5,1991-05-03,6899
4,1991,6,1991-06-07,7220


In [97]:
gas_df['Date'] = gas_df.apply(lambda x: datetime(x['Year'], x['Month'], 1), axis=1)
gas_df = gas_df.drop("Year", axis=1)
gas_df = gas_df.drop("Month", axis=1)
gas_df['Date'] =  pd.to_datetime(gas_df['Date'])
gas_df = gas_df.groupby(gas_df['Date']).min()
gas_df.head()

,gasVolume
Date,
1991-02-01,6433
1991-03-01,6777
1991-04-01,6916
1991-05-01,6899
1991-06-01,7220


In [98]:
# Source 2: Diesel supplied per week.
# Source 2: Calling the API
url = "http://api.eia.gov/series/?api_key=d13b79ac20ecbc443a35c43f026bbd48&series_id=PET.WDIUPUS2.W"
diesel = requests.get(url).json()
diesel

diesel_df = pd.json_normalize(diesel, ['series',['data']],  errors='ignore', max_level=1)
diesel_df.columns = ['Date','dieselVolume']
diesel_df

,Date,dieselVolume
0,20201009,4175
1,20201002,3868
2,20200925,3655
3,20200918,3959
4,20200911,2809
...,...,...
1544,19910308,3338
1545,19910301,2964
1546,19910222,2847
1547,19910215,3121


In [99]:
diesel_df['Date'] =  pd.to_datetime(diesel_df['Date'])
diesel_df['Year'] = diesel_df.Date.dt.year
diesel_df['Month'] = diesel_df.Date.dt.month
diesel_df.head()

,Date,dieselVolume,Year,Month
0,2020-10-09,4175,2020,10
1,2020-10-02,3868,2020,10
2,2020-09-25,3655,2020,9
3,2020-09-18,3959,2020,9
4,2020-09-11,2809,2020,9


In [100]:
diesel_df['Date'] = diesel_df.apply(lambda x: datetime(x['Year'], x['Month'], 1), axis=1)
diesel_df = diesel_df.drop("Year", axis=1)
diesel_df = diesel_df.drop("Month", axis=1)
diesel_df['Date'] =  pd.to_datetime(diesel_df['Date'])
diesel_df = diesel_df.groupby(diesel_df['Date']).min()
diesel_df.head()

,dieselVolume
Date,
1991-02-01,2847
1991-03-01,2946
1991-04-01,2610
1991-05-01,2346
1991-06-01,2675


In [101]:
# Source 3: Jet Fuel supplied per week.
# Source 3: Calling the API
url = "http://api.eia.gov/series/?api_key=d13b79ac20ecbc443a35c43f026bbd48&series_id=PET.WKJUPUS2.W"
jetfuel = requests.get(url).json()
jetfuel

jet_df = pd.json_normalize(jetfuel, ['series',['data']],  errors='ignore', max_level=1)
jet_df.columns = ['Date','jetVolume']
jet_df

,Date,jetVolume
0,20201009,1172
1,20201002,907
2,20200925,858
3,20200918,935
4,20200911,947
...,...,...
1544,19910308,1402
1545,19910301,1514
1546,19910222,1332
1547,19910215,1226


In [102]:
jet_df['Date'] =  pd.to_datetime(jet_df['Date'])
jet_df['Year'] = jet_df.Date.dt.year
jet_df['Month'] = jet_df.Date.dt.month
jet_df.head()

,Date,jetVolume,Year,Month
0,2020-10-09,1172,2020,10
1,2020-10-02,907,2020,10
2,2020-09-25,858,2020,9
3,2020-09-18,935,2020,9
4,2020-09-11,947,2020,9


In [103]:
jet_df['Date'] = jet_df.apply(lambda x: datetime(x['Year'], x['Month'], 1), axis=1)
jet_df = jet_df.drop("Year", axis=1)
jet_df = jet_df.drop("Month", axis=1)
jet_df['Date'] =  pd.to_datetime(jet_df['Date'])
jet_df = jet_df.groupby(jet_df['Date']).min()
jet_df.head()

,jetVolume
Date,
1991-02-01,1226
1991-03-01,1252
1991-04-01,1184
1991-05-01,1096
1991-06-01,1041


In [104]:
fuel_supplied_df = pd.merge((pd.merge(gas_df, diesel_df, how='inner', on='Date')), jet_df, how='inner', on='Date')

In [105]:
fuel_supplied_df

,gasVolume,dieselVolume,jetVolume
Date,,,
1991-02-01,6433,2847,1226
1991-03-01,6777,2946,1252
1991-04-01,6916,2610,1184
1991-05-01,6899,2346,1096
1991-06-01,7220,2675,1041
...,...,...,...
2020-06-01,7870,3302,588
2020-07-01,8550,3019,924
2020-08-01,8630,3253,940


In [106]:
fuel_supplied_df = fuel_supplied_df.loc['2015-10-1' : '2020-10-1']
fuel_supplied_df.head()

,gasVolume,dieselVolume,jetVolume
Date,,,
2015-10-01,8958,3618,1420
2015-11-01,8921,3646,1519
2015-12-01,9185,3268,1396
2016-01-01,8159,2832,1340
2016-02-01,9121,3162,1490


In [107]:
engine = create_engine('sqlite:///fuelsupplied_db.db', echo=True)
sqlite_connection = engine.connect()

2020-10-16 21:39:29,598 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-10-16 21:39:29,602 INFO sqlalchemy.engine.base.Engine ()
2020-10-16 21:39:29,605 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-10-16 21:39:29,607 INFO sqlalchemy.engine.base.Engine ()


In [108]:
fuel_supplied_df.to_sql('users', con=engine, if_exists='append')
engine.execute("SELECT * FROM users").fetchall()

2020-10-16 21:39:30,726 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("users")
2020-10-16 21:39:30,727 INFO sqlalchemy.engine.base.Engine ()
2020-10-16 21:39:30,735 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-10-16 21:39:30,740 INFO sqlalchemy.engine.base.Engine INSERT INTO users ("Date", "gasVolume", "dieselVolume", "jetVolume") VALUES (?, ?, ?, ?)
2020-10-16 21:39:30,741 INFO sqlalchemy.engine.base.Engine (('2015-10-01 00:00:00.000000', 8958, 3618, 1420), ('2015-11-01 00:00:00.000000', 8921, 3646, 1519), ('2015-12-01 00:00:00.000000', 9185, 3268, 1396), ('2016-01-01 00:00:00.000000', 8159, 2832, 1340), ('2016-02-01 00:00:00.000000', 9121, 3162, 1490), ('2016-03-01 00:00:00.000000', 9244, 3373, 1364), ('2016-04-01 00:00:00.000000', 9224, 3609, 1468), ('2016-05-01 00:00:00.000000', 9516, 3827, 1431)  ... displaying 10 of 61 total bound parameter sets ...  ('2020-09-01 00:00:00.000000', 8390, 2809, 858), ('2020-10-01 00:00:00.000000', 8576, 3868, 907))
2020-10-1

OperationalError: (sqlite3.OperationalError) table users has no column named Date
[SQL: INSERT INTO users ("Date", "gasVolume", "dieselVolume", "jetVolume") VALUES (?, ?, ?, ?)]
[parameters: (('2015-10-01 00:00:00.000000', 8958, 3618, 1420), ('2015-11-01 00:00:00.000000', 8921, 3646, 1519), ('2015-12-01 00:00:00.000000', 9185, 3268, 1396), ('2016-01-01 00:00:00.000000', 8159, 2832, 1340), ('2016-02-01 00:00:00.000000', 9121, 3162, 1490), ('2016-03-01 00:00:00.000000', 9244, 3373, 1364), ('2016-04-01 00:00:00.000000', 9224, 3609, 1468), ('2016-05-01 00:00:00.000000', 9516, 3827, 1431)  ... displaying 10 of 61 total bound parameter sets ...  ('2020-09-01 00:00:00.000000', 8390, 2809, 858), ('2020-10-01 00:00:00.000000', 8576, 3868, 907))]
(Background on this error at: http://sqlalche.me/e/e3q8)